## IMPORTS

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import random
import os, sys, time
from PIL import Image
import torchvision.transforms as T
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.ndimage as sc
from PIL import ImageFilter

sys.path.append('/content/drive/MyDrive/ml_epfl/ml_road_segmentation/script/')
from models import *
from helper_functions import *
from dataset_loading import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CONSTANTS

In [9]:
CHECKPOINT_PATH ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/current_checkpoint.pt"
BEST_MODEL_PATH ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/best_model.pt"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NBR_EPOCHS = 100
BATCH_SIZE = 1
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-6
GAMMA = 1
K_FOLD = 4
VALIDATION_SET_IDX = 0
BATCH_SIZE_VAL = 5
DIM_IMG_TRAIN = 400

FILTER_1 = ImageFilter.FIND_EDGES
FILTER_2 = ImageFilter.CONTOUR

## TRAIN WITH K-FOLDS (FOR BAGGING)...

In [ ]:
K_FOLD_ROOT_PATH_LOADING = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_checkpoints/"
K_FOLD_ROOT_PATH_WRITTING = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_bagging/"

# Constants K-Fold
K_FOLD = 4
LEARNING_RATE_K_FOLD = 1e-3
WEIGHT_DECAY_K_FOLD = 1e-6
MOMENTUM_SGD_K_FOLD = 0.99
GAMMA_K_FOLD = 1
SEED_K_FOLD = 0
NBR_EPOCHS_K_FOLD = 30
BATCH_SIZE_VAL_K_FOLD = 5
BATCH_SIZE_K_FOLD = 1
FILTER = ImageFilter.FIND_EDGES
OPTIMIZERS = ["SGD_"]
ANGLE_ROT = 9
NB_ROT = 1

models = []
for opti_name in OPTIMIZERS:
    for idx_validaton_set in range(K_FOLD): 
        print(f'Optimizer {opti_name[:-1]} - fold {idx_validaton_set}')
        #defining paths
        CHECKPOINT_K_FOLD_PATH = K_FOLD_ROOT_PATH_WRITTING + opti_name + 'fold_' + str(idx_validaton_set) +'_all_data_bagging_from_zero.pt' 
        BEST_MODEL_K_FOLD_PATH = K_FOLD_ROOT_PATH_WRITTING + opti_name + 'fold_' + str(idx_validaton_set) +'_model_bagging_from_zero.pt'

        model =  u_net().to(DEVICE)
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE_K_FOLD, weight_decay=WEIGHT_DECAY_K_FOLD, momentum=MOMENTUM_SGD_K_FOLD) 
        scheduler = ExponentialLR(optimizer, GAMMA_K_FOLD)
        criterion = nn.CrossEntropyLoss()

        #loading dataset
        dataset = imagesDataset(K_FOLD, idx_validaton_set, BATCH_SIZE_VAL_K_FOLD, SEED_K_FOLD, FILTER, ANGLE_ROT, NB_ROT)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE_K_FOLD, shuffle = True)
        validation_dataloader = dataset.get_validation_dataloader()

        #resuming training
        train(NBR_EPOCHS_K_FOLD, train_loader, model, optimizer, scheduler, criterion, DEVICE, CHECKPOINT_K_FOLD_PATH, BEST_MODEL_K_FOLD_PATH, f1_init = 0)
        models.append(model)

## .... OR LOAD EXISTING MODEL

In [ ]:
def load_k_models():
    models_list = []
    for idx_model in range(4):
        K_FOLD_MODEL_PATH = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_bagging/SGD_fold_" + str(idx_model) +'_model_bagging.pt'
        model =  u_net()
        model.load_state_dict(torch.load(K_FOLD_MODEL_PATH))
        models_list.append(model)
    return models_list

models = load_k_models()

# MAKE PREDICTIONS

In [ ]:
def get_bagging_prediction(input, models_list, prediction_rule):
    assert len(models_list) == 4
    predictions = []
    for model in models_list:
        model.eval()
        predictions.append(model(input))
    return prediction_rule(predictions)

models = load_k_models()

In [ ]:
def append_filter(filter_to_apply, images):
    to_tensor = T.ToTensor() 
    toPIL = T.ToPILImage()
    
    concatenated_images = []

    for img in images:
      filtered_img = toPIL(img).filter(filter_to_apply)
      tensored_img = to_tensor(filtered_img)
      #stacked_img = torch.stack((img, tensored_img), dim=1)
      stacked_img = torch.cat((img, tensored_img), dim=0)
      concatenated_images.append(stacked_img)
    
    return concatenated_images

def load_test_dataset():
    root_test_dir = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/data/test_set_images/"
    test_i = 'test_'
    n = len(os.listdir(root_test_dir))
    test_imgs = [to_tensor(Image.open(root_test_dir + test_i + str(idx_img) +'/'+ test_i + str(idx_img) + '.png')) for idx_img in range(1, n+1)]
    test_imgs = append_filter(FILTER_1, test_imgs)
   
    test_imgs = [img.reshape((1, NB_CHANNELS_INIT, DIM_IMG_TEST, DIM_IMG_TEST)) for img in test_imgs]
    return test_imgs
  
test_images = load_test_dataset()